In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import datetime
# os.chdir("../")


In [2]:
def last_day_of_month(row):
    if row['month'] == 12:
        next_month = 1
        next_year = row['year'] + 1
    else:
        next_month = row['month'] + 1
        next_year = row['year']
    last_day = datetime.datetime(next_year, next_month, 1) - datetime.timedelta(days=1)
    return last_day

In [3]:
data=pd.read_csv("../merged/len3_prod_mon.csv",encoding="shift-jis",index_col=0)
# drop never treated group
count=0
data=data.dropna(subset=["release_year"])
data["r_date"]=pd.to_datetime(data["date"],format="%Y/%m/%d")
data["r_date"]
data["prod_date"]=data.apply(lambda row: last_day_of_month(row),axis=1)

C:\Users\hakos\AppData\Local\Temp\ipykernel_3104\3350463267.py:1: DtypeWarning: Columns (16,17,19,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len3_prod_mon.csv",encoding="shift-jis",index_col=0)


In [4]:
data["elapsed"]=data["prod_date"]-data["r_date"]
data["elapsed"]=data["elapsed"].dt.days//91
data["elapsed"]

36       56
37       56
38       56
39       56
40       56
         ..
39640    48
39753    85
39754    86
39755    86
39756    86
Name: elapsed, Length: 16113, dtype: int64

1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [5]:
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
if count:
    # count ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"month"]).first(), 
        dummies.groupby([data['year'],data['id'],data["month"]]).sum()
    ], axis=1).reset_index()
else: 
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"month"]).first(), 
        dummies.groupby([data['year'],data['id'],data["month"]]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
aggregated_df

,year,id,month,efficacy_x,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,elapsed_124,elapsed_125,elapsed_126,elapsed_127,elapsed_128,elapsed_129,elapsed_130,elapsed_131,elapsed_132,elapsed_133
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,11234470,11067590,8160160,2586928,2581213,5715,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,12935859,12691332,10524766,2549867,2549642,225,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,7769934,7731996,5017852,2500236,2500236,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4462,2023,629,4,その他の化学療法剤,"3,835,795","2,721,078","823,165",-,-,-,...,0,0,0,0,0,0,0,0,0,0
4463,2023,799,1,他に分類されない治療を主目的としない医薬品,"1,346,611","1,346,611","653,044","204,097","204,097","116,625",...,0,0,0,0,0,0,0,0,0,0
4464,2023,799,2,他に分類されない治療を主目的としない医薬品,"1,455,085","1,455,085","820,819","248,563","248,328","127,352",...,0,0,0,0,0,0,0,0,0,0
4465,2023,799,3,他に分類されない治療を主目的としない医薬品,"1,688,308","1,688,308","913,299","580,453","193,456",-,...,0,0,0,0,0,0,0,0,0,0


In [6]:
colnames

['elapsed_m55',
 'elapsed_m54',
 'elapsed_m53',
 'elapsed_m52',
 'elapsed_m51',
 'elapsed_m50',
 'elapsed_m49',
 'elapsed_m48',
 'elapsed_m47',
 'elapsed_m46',
 'elapsed_m45',
 'elapsed_m44',
 'elapsed_m43',
 'elapsed_m42',
 'elapsed_m41',
 'elapsed_m40',
 'elapsed_m39',
 'elapsed_m38',
 'elapsed_m37',
 'elapsed_m36',
 'elapsed_m35',
 'elapsed_m34',
 'elapsed_m33',
 'elapsed_m32',
 'elapsed_m31',
 'elapsed_m30',
 'elapsed_m29',
 'elapsed_m28',
 'elapsed_m27',
 'elapsed_m26',
 'elapsed_m25',
 'elapsed_m24',
 'elapsed_m23',
 'elapsed_m22',
 'elapsed_m21',
 'elapsed_m20',
 'elapsed_m19',
 'elapsed_m18',
 'elapsed_m17',
 'elapsed_m16',
 'elapsed_m15',
 'elapsed_m14',
 'elapsed_m13',
 'elapsed_m12',
 'elapsed_m11',
 'elapsed_m10',
 'elapsed_m9',
 'elapsed_m8',
 'elapsed_m7',
 'elapsed_m6',
 'elapsed_m5',
 'elapsed_m4',
 'elapsed_m3',
 'elapsed_m2',
 'elapsed_m1',
 'elapsed_0',
 'elapsed_1',
 'elapsed_2',
 'elapsed_3',
 'elapsed_4',
 'elapsed_5',
 'elapsed_6',
 'elapsed_7',
 'elapsed_8',
 'e

connect generic usage

In [7]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len3_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric

,薬効分類,year,generic_share_q,generic_share_r
0,114.0,2008.0,0.223357,0.058586
1,114.0,2009.0,0.223357,0.058586
2,114.0,2010.0,0.223357,0.058586
3,114.0,2011.0,0.223357,0.058586
4,114.0,2012.0,0.227087,0.058586
...,...,...,...,...
395,799.0,2019.0,0.006196,0.024777
396,799.0,2020.0,0.006056,0.022847
397,799.0,2021.0,0.007953,0.028184
398,799.0,2022.0,0.028110,0.043253


In [8]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["薬効分類","year"])
merged_df

,year,id,month,efficacy_x,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,elapsed_129,elapsed_130,elapsed_131,elapsed_132,elapsed_133,Year,Percentage,薬効分類,generic_share_q,generic_share_r
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
1,2009,114,2,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
2,2009,114,3,解熱鎮痛消炎剤,11234470,11067590,8160160,2586928,2581213,5715,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
3,2009,114,4,解熱鎮痛消炎剤,12935859,12691332,10524766,2549867,2549642,225,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
4,2009,114,5,解熱鎮痛消炎剤,7769934,7731996,5017852,2500236,2500236,0,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,2022,799,8,他に分類されない治療を主目的としない医薬品,1416628,1416628,748860,308849,308849,-,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253
4195,2022,799,9,他に分類されない治療を主目的としない医薬品,1899785,1598049,658356,391420,283334,-,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253
4196,2022,799,10,他に分類されない治療を主目的としない医薬品,1496351,1496351,655730,417711,335149,-,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253
4197,2022,799,11,他に分類されない治療を主目的としない医薬品,1325263,1325263,794279,209154,200543,-,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253


In [9]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,month,efficacy_x,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,id_264,id_265,id_332,id_339,id_399,id_441,id_449,id_625,id_629,id_799
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,11234470,11067590,8160160,2586928,2581213,5715,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,12935859,12691332,10524766,2549867,2549642,225,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,7769934,7731996,5017852,2500236,2500236,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,2022,799,8,他に分類されない治療を主目的としない医薬品,1416628,1416628,748860,308849,308849,-,...,0,0,0,0,0,0,0,0,0,1
4195,2022,799,9,他に分類されない治療を主目的としない医薬品,1899785,1598049,658356,391420,283334,-,...,0,0,0,0,0,0,0,0,0,1
4196,2022,799,10,他に分類されない治療を主目的としない医薬品,1496351,1496351,655730,417711,335149,-,...,0,0,0,0,0,0,0,0,0,1
4197,2022,799,11,他に分類されない治療を主目的としない医薬品,1325263,1325263,794279,209154,200543,-,...,0,0,0,0,0,0,0,0,0,1


In [10]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if not(count):
    print("dummy")
    # dummy ver.
    df_with_dummies.to_csv("../merged/len3_prod_mon_Q_DN.csv",encoding="shift-jis")
else:    
    # count ver.
    df_with_dummies.to_csv("../merged/len3_prod_mon_Q_sum_DN.csv",encoding="shift-jis")

dummy
